In [1]:
#!unzip robohearts.zip
%cd robohearts/

/content/robohearts


In [0]:
import gym
from gymhearts.Hearts import *
from mc import MC
from gymhearts.Agent.human import Human
from gymhearts.Agent.randomAI import RandomAI

In [0]:
NUM_ITERS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100
playersNameList = ['MC', 'Rando', 'Megumin', 'Darkness']
agent_list = [0, 0, 0, 0]

# Human vs Random
agent_list[0] = MC(playersNameList[0])
agent_list[1] = RandomAI(playersNameList[1])
agent_list[2] = RandomAI(playersNameList[2])
agent_list[3] = RandomAI(playersNameList[3])

In [0]:
%%capture 
# who cares about output in training
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
for trn_episode in range(TRAINING_ITERS):
    observation = env.reset()
    history = []
    while True:
        now_event = observation['event_name']
        IsBroadcast = observation['broadcast']
        action = None
        if IsBroadcast == True:
            for agent in agent_list:
                agent.Do_Action(observation)
        else:
            playName = observation['data']['playerName']
            for agent in agent_list:
                if agent.name == playName:
                    action = agent.Do_Action(observation)

        # update my agent
        if now_event == 'RoundEnd':
            agent_list[0].update_reward_fn(history, -reward['MC'])
            history = []
            weights = agent_list[0].weight_vec
        if now_event == 'GameOver':
              break     
        observation2, reward, done, info = env.step(action)
        if not IsBroadcast and observation['data']['playerName'] == 'MC':
            history.append(observation)
        observation = observation2


In [0]:
%%capture
# Evaluate the MC!
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MC(playersNameList[0], params={'weights' : weights})
mc_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MC')
              break
  mc_wins.append(num_won)

In [0]:
%%capture
# Evaluate the Random!
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = RandomAI(playersNameList[0])
rand_wins = []
for i in range(NUM_ITERS):
  num_won = 0
  for i_episode in range(NUM_EPISODES):
      observation = env.reset()
      while True:
          now_event = observation['event_name']
          IsBroadcast = observation['broadcast']
          action = None
          if IsBroadcast == True:
              for agent in agent_list:
                  agent.Do_Action(observation)
          else:
              playName = observation['data']['playerName']
              for agent in agent_list:
                  if agent.name == playName:
                      action = agent.Do_Action(observation)
          if now_event == 'GameOver':
              num_won += int(observation['data']['Winner'] == 'MC')
              break
  rand_wins.append(num_won)

In [0]:
print("The weights are: " + str(weights))
print("MC won: " + str(mc_wins) + " times")
print("Rand won: " + str(rand_wins) + " times")